# Validation of surface PFTs using point observations

In [ ]:
# bin_value using function from r4ecology's github
import numpy as np
def bin_value(x, bin_res):
    return np.floor((x + bin_res / 2) / bin_res + 0.5) * bin_res - bin_res / 2



In [ ]:
import nctoolkit as nc
nc.options(parallel=True)
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Markdown as md
%load_ext rpy2.ipython
import jellyfish
import ecoval
import copy
import calendar
data_dir = ecoval.get_datadir(level = -2)
import nctoolkit as nc
import hvplot.xarray
import geopandas as gpd
import xarray as xr
import holoviews as hv
from holteandtalley import HolteAndTalley
import numpy as np
from mask import mask_all, mask_shelf
import pandas as pd
import glob
import cmocean as cm
from tqdm import tqdm
import hvplot.pandas
from plotnine import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
i_figure = 1
i_table = 1
stamp = nc.session_info["stamp"]
out = ".trackers/" + stamp
fast_plot = False
if not os.path.exists(".trackers"):
    os.makedirs(".trackers")
# save out as empty file
with open(out, 'w') as f:
    f.write("")

In [ ]:
variable = "pft".lower()
vv_name = variable
if vv_name.lower() == "ph":
    vv_name = "pH"
if vv_name in ["doc", "poc"]:
    vv_name = vv_name.upper()
if vv_name == "benbio":
    vv_name = "biomass of macrobenthos"
layer = "surface"
# get the units. File inspection could be randomized in case people have put loose files in there...
import glob
df = pd.read_csv("../../matched/mapping.csv")
if variable == "pft":
    df = df.query("variable == 'chlorophyll'")
else:
    df = df.query("variable == @variable")
pattern = list(df.pattern)[0]
paths = pd.read_csv(glob.glob(f"../../matched/point/**/{layer}/{variable}/paths.csv")[0]).path

for ff in paths:
    try:
        ds = nc.open_data(paths[0])
        model_variable = list(df.model_variable)[0].split("+")[0]
        unit = list(ds.contents.query("variable == @model_variable").unit)[0]
        break
    except:
        pass

## Read in the data

In [ ]:
ff = glob.glob(f"../../matched/point/**/{layer}/{variable}/*_{variable}.csv")[0]
vv_source = os.path.basename(ff).split("_")[0]
vv_source = vv_source.upper()
df = pd.read_csv(ff)
if variable == "ph":
    df = df.query("observation > 4").reset_index(drop = True)
# Danish part is always dubious
df = df.query("lon < 9")
# ds= nc.open_data(f"{data_dir}/amm7_val_subdomains.nc")
# ds.subset(variable = "Shelf")
# ds.as_missing(0)
# ds.regrid(df.loc[:,["lon", "lat"]].drop_duplicates().reset_index(drop = True), "nn")
# df_grid = ds.to_dataframe().reset_index().dropna().drop_duplicates()
# df = df.merge(df_grid)
df_locs = df.loc[:,["lon", "lat"]].drop_duplicates()
# bin to 0.01 resolution
df_raw = df
df["lon"] = df["lon"].apply(lambda x: bin_value(x, 0.5))
df["lat"] = df["lat"].apply(lambda x: bin_value(x, 0.5))
if variable == "benbio":
    df = df.assign(observation = lambda x: 1000 * 0.45 * x.observation) 
if variable not in  ["carbon", "benbio"]:
    df = df.groupby(["lon", "lat", "year", "month"]).mean().reset_index()
else:
    df = df.groupby(["lon", "lat"]).mean().reset_index()

In [ ]:
if variable == "carbon":
    md("**Note**: This is in progress. Model and observation data are yet to be converted to comparable units!")

In [ ]:
from IPython.display import Markdown as md

if vv_source == "ices": 

    if layer == "bottom":
        md(f"Near-bottom values of {vv_name} were extracted from ICES bottle and CTD data.")
    if layer == "surface":
        md(f"Values from the top 5 m of the water column were extracted from ICES bottle and CTD data.")
    if layer == "benthic":
        md("Benthic values were extracted from existing datasets")


if layer == "bottom":
    md(f"This data was extracted from vertical profiles. The near_bottom value was defined as the value closest to the bottom, that was within 5 m of the bottom. Bathymetry was estimated using GEBCO Bathymetry data.")
if layer == "surface":
    md(f"This data was extracted from vertical profiles. Values from the top 5 m were extracted from the database. This was compared with the model values from the surface level.")
if variable in ["benbio"]:
    md("Biomass data for macrobenthos was downloaded from the North Sea Benthos Survey 1986.")

if variable in ["carbon"]:
    md("Carbon data was compiled from multiple sources")
md(f"In total there were {len(df)} {layer} values extracted from the observational database.")

if layer == "bottom":
    md("**Note:** this analysis has been restricted to observations on the shelf region.")


if variable == "poc":
    md("Particulate organic carbon data was compiled from multiple sources")

if variable == "pco2":
    md("The variable pCO2water_SST_wet was extracted from the SOCAT 2023 database.")
    md("Observational values were averaged for each day in the year.")

if variable == "doc":
    md("Dissolved organic carbon data was compiled from multiple sources")

df_mapping = pd.read_csv("../../matched/mapping.csv")

In [ ]:
%%capture --no-display
%%R -i df_locs -i variable -i unit -w 1000 -h 1200
library(dplyr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(stringr)
world_map <- map_data("world")
# get lon, lat limits from profile_mld

xlim = c(min(df_locs$lon), max(df_locs$lon))
ylim = c(min(df_locs$lat), max(df_locs$lat))



gg <- df_locs %>%
# final six months of the year
    ggplot()+
    geom_point(aes(lon, lat))+
    theme_gray(base_size = 24)+
    # add colour scale. Minimum zero, label 100, ">100"
    geom_polygon(data = world_map, aes(long, lat, group = group), fill = "grey60")+
    coord_fixed(xlim = xlim, ylim = ylim, ratio = 1.5) 

# figure out if lon minimum is less than -10
if( min(df_locs$lon) < -10 ){
    # add sensible labels for longitude and latitude

    gg <- gg +
    scale_x_continuous(breaks = seq(-10, 5, 5), labels = c("10°W", "5°W", "0°", "5°E"))+ 
    scale_y_continuous(breaks = seq(45, 60, 5), labels = c("45°N", "50°N", "55°N", "60°N"))+
    labs(x = "", y = "") 


}

    # move legen

gg

In [ ]:
md(f"**Figure {chapter}{i_figure}:** Map of PFT observations from Cefas.")
i_figure = i_figure + 1

In [ ]:
#df
# melt columns with model in the name
if variable == "pft":
    df = (
        df
        .melt(id_vars = ["lon", "lat", "year", "month"], var_name = "measure", value_name = "value")
    
    
    )
df["source"] = [x.split("_")[-1] for x in df.measure]
df = (df
        .assign(measure = lambda x: x.measure.str.replace("_model", ""))
        .assign(measure = lambda x: x.measure.str.replace("_obs", ""))
)

In [ ]:
# pivot value using measure
df = (
    df.pivot_table(index = ["lon", "lat", "year", "month", "source"], columns = "measure", values = "value").reset_index()
    .reset_index(drop = True)
    # drop index name
    .rename_axis(None, axis = 1)
)
# change mod to model and obs  to observation in source
df["source"] = df.source.str.replace("model", "Model").str.replace("obs", "Observation")

In [ ]:
%%capture --no-display
%%R -i df -w 1000 -h 500

library(ggtern)
library(tidyverse)

ggtern(data = df, aes(x = micro_frac, y = pico_frac, z = nano_frac)) +
    geom_point()+
    theme_rgbw(base_size = 24)+
    facet_wrap(~source)+
    # add better labels
    theme(legend.position = "bottom")+
    labs( x = "Micro", y = "Pico", z = "Nano")

In [ ]:
i_subplot = 1
md(f"**Figure {chapter}{i_figure}.{i_subplot}**: Ternary plot of the model output for Plankton Functional Types.")
i_subplot += 1

In [ ]:
%%capture --no-display
%%R -i df -w 1000 -h 800

library(ggtern)
library(tidyverse)

ggtern(data = df, aes(x = micro_frac, y = pico_frac, z = nano_frac, colour = source)) +
    geom_point()+
    theme_rgbw(base_size = 24)+
    # add better labels
    theme(legend.position = "bottom")+
    labs( x = "Micro", y = "Pico", z = "Nano")+
    # ditch legend labels
    theme(legend.title = element_blank())

In [ ]:
md(f"**Figure {i_figure}.{i_subplot}**: Ternary plot of the model output for Plankton Functional Types.")
i_subplot += 1

## Data Sources

Creach and Forster (2017). North Sea phytoplankton pigments 2010 to 2011. Cefas, UK. V1. doi: https://doi.org/10.14466/CefasDataHub.33.